In [2]:
# Imports
import numpy as np
import pandas as pd
# import dcMinMaxFunctions as dc
# import dcor
from scipy.misc import derivative
from sklearn.model_selection import train_test_split
import math

import torch
from scipy import stats
import wandb
from cov_help import *



In [6]:
data_path = 'data/covtype.csv'
norm =1
X,Y = cov_data_loader(data_path,norm=norm)


In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
train_priv = torch.utils.data.TensorDataset(X_train, Y_train)
test_priv = torch.utils.data.TensorDataset(X_test, Y_test)
trainloader_priv = torch.utils.data.DataLoader(train_priv, batch_size=4096,
                                          shuffle=True, num_workers=4, drop_last=True)
testloader_priv = torch.utils.data.DataLoader(test_priv, batch_size=4096,
                                          shuffle=False, num_workers=4, drop_last=True)

In [8]:
net = Net_new(1)

In [3]:
model_path = 'Models/cov_full_2'
model_path.replace('Models/','')

'cov_full_2'

In [3]:
from cov_help import *
data_path = 'data/covtype.csv'
norm =1
X,Y = cov_data_loader(data_path,norm=norm)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_public =  X_train[::10]
# X_private  = torch.load('Embeddings/cov_full/X_emb_test.pt')[0::10]

In [4]:
X_public.shape

torch.Size([46481, 54])

In [34]:
net = Net_new(1,device = torch.device("cuda:0"))
state_dict  = torch.load("Models/cov_non_priv_emb_512_100")

net.load_state_dict(state_dict)


<All keys matched successfully>

In [35]:
X_priv_real  = X_train[1::10]
train_priv = torch.utils.data.TensorDataset(X_priv_real, Y_train[1::10])


trainloader_priv = torch.utils.data.DataLoader(train_priv, batch_size=4096,
                                    shuffle=False, num_workers=4)


X_emb_train,losses_train = create_model_embs2(net,trainloader_priv,device= torch.device("cuda"),l=len(X_priv_real),h=0.82)


100%|██████████| 12/12 [00:25<00:00,  2.10s/it]


In [7]:
X_private = X_emb_train

In [8]:
import torch
from torch.utils.data import Dataset

Combined_dataset = torch.utils.data.TensorDataset(X_private,X_public)
data_loader = torch.utils.data.DataLoader(Combined_dataset, batch_size=4096, shuffle=True)


In [36]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the generator model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # Define the layers of the encoder
        self.encoder = nn.Sequential(
            nn.Linear(54, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 54)
        )
        self.decoder = nn.Sequential(
            nn.Linear(54, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 54)
        )

    def forward(self, x):
        # Implement the forward pass of the generator
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat

# Define the discriminator model
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(54, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
        # Define the layers of the discriminator

    def forward(self, x):
        # Implement the forward pass of the discriminator
        z = self.model(x)
        return z

# Initialize the generator and discriminator models
ae = Autoencoder()
discriminator = Discriminator()

# Define the loss function and optimizer for the generator and discriminator
loss_function_disc = nn.BCELoss()
loss_function_ae = nn.MSELoss()
ae_optimizer = optim.Adam(ae.parameters(), lr=0.001)
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for batch_idx, (private_data, public_data) in enumerate(data_loader):
        batch_size = public_data.size(0)

        # Train the discriminator
        discriminator.zero_grad()
        public_labels = torch.ones(batch_size, 1)
        private_labels = torch.zeros(batch_size, 1)

        # Forward pass real data through discriminator
        private_output = discriminator(private_data)
        private_loss = loss_function_disc(private_output, private_labels)

        # Generate fake data using the generator
        # z = torch.randn(batch_size, latent_dim)
        public_embedding = ae.encoder(public_data)
        public_data_rec = ae.decoder(public_embedding)

        # Forward pass fake data through discriminator
        public_output = discriminator(public_embedding.detach())
        public_loss = loss_function_disc(public_output, public_labels)

        # Total discriminator loss
        discriminator_loss = public_loss + private_loss
        discriminator_loss.backward()
        discriminator_optimizer.step()

        # Train the generator
        ae.zero_grad()
        
        
        # Forward pass fake data through discriminator
        
        ae_loss = loss_function_ae(public_data_rec, public_data)

        ae_loss.backward()
        ae_optimizer.step()

        # Print the loss for every few batches
        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch}/{num_epochs}], Batch [{batch_idx}/{len(data_loader)}], "
                  f"Discriminator Loss: {discriminator_loss.item():.4f}, "
                  f"Autoencoder Loss: {ae_loss.item():.4f}")

# Save the trained generator and discriminator models
# torch.save(generator.state_dict(), 'generator.pth')
# torch.save(discriminator.state_dict(), 'discriminator.pth')
# X[10]

Epoch [0/10], Batch [0/12], Discriminator Loss: 1.3876, Autoencoder Loss: 0.0065
Epoch [1/10], Batch [0/12], Discriminator Loss: 1.2185, Autoencoder Loss: 0.0014
Epoch [2/10], Batch [0/12], Discriminator Loss: 0.9313, Autoencoder Loss: 0.0010
Epoch [3/10], Batch [0/12], Discriminator Loss: 0.5676, Autoencoder Loss: 0.0008
Epoch [4/10], Batch [0/12], Discriminator Loss: 0.2750, Autoencoder Loss: 0.0007
Epoch [5/10], Batch [0/12], Discriminator Loss: 0.1083, Autoencoder Loss: 0.0004
Epoch [6/10], Batch [0/12], Discriminator Loss: 0.0394, Autoencoder Loss: 0.0003
Epoch [7/10], Batch [0/12], Discriminator Loss: 0.0170, Autoencoder Loss: 0.0003
Epoch [8/10], Batch [0/12], Discriminator Loss: 0.0093, Autoencoder Loss: 0.0003
Epoch [9/10], Batch [0/12], Discriminator Loss: 0.0060, Autoencoder Loss: 0.0002


In [37]:
x1 = ae.decoder(X_private) 
x2 = X_priv_real

In [38]:
x1_np = x1.detach().numpy()
x2_np = x2.detach().numpy()
total = 0 
for i in range(len(x1_np)):
    correlation = np.corrcoef(x1_np[i], x2_np[i])[0][1]
    total += correlation
print(total/len(x1_np))

0.8578848931918251


In [39]:
import pandas as pd
df = pd.read_csv('data/covtype.csv')

In [40]:
torch.max(x2[0][-40:])

tensor(0.0001)

In [41]:
x2[0][-40:]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0001, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000])

In [42]:
x2

tensor([[0.3164, 0.0022, 0.0019,  ..., 0.0000, 0.0000, 0.0000],
        [0.2667, 0.0086, 0.0005,  ..., 0.0000, 0.0000, 0.0000],
        [0.3355, 0.0234, 0.0010,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.3202, 0.0023, 0.0017,  ..., 0.0000, 0.0000, 0.0000],
        [0.3358, 0.0035, 0.0009,  ..., 0.0000, 0.0000, 0.0000],
        [0.3309, 0.0256, 0.0017,  ..., 0.0000, 0.0000, 0.0000]])

In [43]:
sum = 0
for i in range(len(x2)):
    sum+= (torch.argmax(torch.abs(x1[i][-40:])) == torch.argmax(x2[i][-40:]))
sum/len(x2)

tensor(0.0363)

In [44]:
sum

tensor(1686)

In [45]:
x3 = torch.zeros((x2.shape))
for i in range(len(x2)):
    x3[i][:13] = x1[i][:13]
    x3[i][int(torch.argmax(x1[i][-40:]))] = 0.0001

In [46]:
loss_function_ae(x3, x2)

tensor(0.0008, grad_fn=<MseLossBackward0>)